In [26]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [27]:
df=pd.read_csv('data.csv')
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,80,15.0,98.0,86,high risk
1,35,140,90,13.0,98.0,70,high risk
2,29,90,70,8.0,100.0,80,high risk
3,30,140,85,7.0,98.0,70,high risk
4,35,120,60,6.1,98.0,76,low risk


In [28]:
d = {
    "low risk": 0,
    "mid risk": 1,
    "high risk": 2
}

df['RiskLevel'].replace(to_replace=d, inplace=True)
df.drop(['DiastolicBP'],axis=1)


,Age,SystolicBP,BS,BodyTemp,HeartRate,RiskLevel
0,25,130,15.0,98.0,86,2
1,35,140,13.0,98.0,70,2
2,29,90,8.0,100.0,80,2
3,30,140,7.0,98.0,70,2
4,35,120,6.1,98.0,76,0
...,...,...,...,...,...,...
1009,22,120,15.0,98.0,80,2
1010,55,120,18.0,98.0,60,2
1011,35,85,19.0,98.0,86,2
1012,43,120,18.0,98.0,70,2


In [29]:
x=df.iloc[:,df.columns!='RiskLevel']
Y=df.iloc[:,df.columns=='RiskLevel']
sm=SMOTE()
x_resample,Y_resample=sm.fit_resample(x,Y.values.ravel())
Y_resample=pd.DataFrame(Y_resample)
x_resample=pd.DataFrame(x_resample)
df = pd.concat([x_resample, Y_resample], axis=1)
df.set_axis([*df.columns[:-1], 'RiskLevel'], axis=1)
df.head()

,Age,SystolicBP,DiastolicBP,BS,BodyTemp,HeartRate,0
0,25,130,80,15.0,98.0,86,2
1,35,140,90,13.0,98.0,70,2
2,29,90,70,8.0,100.0,80,2
3,30,140,85,7.0,98.0,70,2
4,35,120,60,6.1,98.0,76,0


In [31]:
X=df.drop(['RiskLevel'],axis=1)
y=df['RiskLevel']

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score



rf = RandomForestClassifier(random_state=42)


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)


grid_search.fit(X, y)

best_params = grid_search.best_params_
best_rf = grid_search.best_estimator_


best_rf.fit(X, y)


y_pred = best_rf.predict(X)


accuracy = accuracy_score(y, y_pred)
print("Tuned Random Forest Accuracy: {:.2f}%".format(accuracy * 100))


print("Best Hyperparameters:", best_params)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Tuned Random Forest Accuracy: 93.43%
Best Hyperparameters: {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}


In [34]:
import joblib
joblib.dump(best_rf, "model.joblib")

['model.joblib']

In [ ]:
loaded_rf = joblib.load("my_random_forest.joblib")